In [ ]:
public class DogLauncher {
    public static void main(String[] args) {
        Dog d1 = new Dog("Elyse", 3);
        Dog d2 = new Dog("Sture", 9);
        Dog d3 = new Dog("Benjamin", 15);
        Dog[] dogs = new Dog[] {d1, d2, d3};
        System.out.println(Maximizer.max(dogs));
        Dog d = (Dog) Maximizer.max(dogs);
        d.bark();
    }
}

In [ ]:
public class Dog implements Comparable<Dog> {
    private String name;
    private int size;
    
    public Dog(String n, int s) {
        name = n;
        size = s;
    }
    
    public void bark() {
        System.out.println(name + " says: bark");
    }
    
    public int compareTo(Dog uddaDog) {
        return this.size - uddaDog.size;
    }
}

# Comparators

We don't always want to compare objects in the same way every time. This is when the `Comparator` comes in.

## Natural Order

The term `Natural Order` is sometimes used to refer to the ordering implied by a `Comparable`'s `compareTo` method.
* Example: `Dog` objects (as we've defined them) have a natural order given by their size (e.g. smallest on the left, largest on the right)

<img src = 'ordering.png' width = 700/>

Sometimes, we might want to order objects in a different way. For example by their name alphabetically.

<img src = 'alphabet.png' width = 500/>

How do we do this in Java?

## Subtype Polymorphism vs. Explicit Higher Order Functions

Suppose we want to write a program that prints a string representation of the larger of 2 objects according to some specific comparison function.

In [ ]:
// Explicit HoF Approach

def print_larger(x, y, compare, stringify):
    if compare(x, y):
        return stringify(x)
    return stringify(y)

In explicit HoF approach, we can simply pass a different `compare` function. The user just has to pick the correct `compare` function.

Now how do we do this in Subtype Polymorphism approach?

In [ ]:
def print_larger(T x, T y):
    if x.largerThan(y):
        return x.str()
    return y.str()

This `print_larger` function takes 2 objects of the same type, calls `largerThan`, and stringify whichever's larger. But how do we augment this code to support multiple orderings?
* The Explicit HoF approach automatically supports multiple ordering since we can simply pass in a different `compare` function

A few potential (but bad) solutions:
1. Have multiple `compareTo` methods (e.g. `compareTo2`, `compareTo3`, so on..)
2. Have a `String` argument to the `compareTo` method like the following,

In [ ]:
public int compareTo(Dog uddaDog, String whichCompare){
    ...
}

However, these solutions won't work well in Java. One solution that works with Java is to pass in an object of type `comparator<T>` 

In [ ]:
def print_larger(T x, T y, comparator<T> c):
    if c.compare(x, y):
        return x.str()
    return y.str()

## Additional Orders in Java

In Java, the `comparator` object can be used as a nested class inside `Dog` class,

In [ ]:
import java.util.Comparator;

public class Dog implements Comparable<Dog> {
    private String name;
    ...
    
    public class NameComparator implements Comparator<Dog> {
        
    }
}

In some languages, we'd write 2 comparison functions and simply pass the one we want:
* `sizeCompare()`
* `nameCompare()`

The standard Java approach: create `sizeComparator` and `nameComparator` class that implements the `Comparator` interface
* Requires methods that also take `Comparator` arguments (see project 1B)

In [ ]:
public interface Comparator<T> {
    int compare(T o1, T o2);
}

With our `NameComparator` class, we'll do the following,

In [ ]:
public class NameComparator implements Comparator<Dog> {
    public int compare(Dog a, Dog b);
    ... // rule of comparison
}

Similar to `compareTo` we want to make comparison rule that:
1. Returns negative number if `a < b`
2. 0 if `a == b`
3. Positive number otherwise

We can use the `String`'s `compareTo` method.

In [ ]:
public class NameComparator implements Comparator<Dog> {
    public int compare(Dog a, Dog b) {
        return a.name.compareTo(b.name);
    }
}

Notice that we don't need to instantiate a `Dog` class to use `NameComparator`. Therefore, we can make the class static.

In [ ]:
public static class NameComparator implements Comparator<Dog> {
    public int compare(Dog a, Dog b) {
        return a.name.compareTo(b.name);
    }
}

Now let's use the `NameComparator` in the `main` method in `DogLauncher.java`. To use `NameComparator`, first we need to create a comparator.

In [ ]:
public static void main(String[] args) {
    ...
    Dog.NameComparator nc = new Dog.NameComparator();
    // nc is now a NameComparator object that's capable of comparing dogs by name
    if (nc.compare(d1, d3) > 0) {
        d1.bark(); // If d1 is greater in alphabet, d1 barks
    } else {
        d3.bark(); // Otherwise d3 barks 
    }
}

The code above works, but we don't often do `new Dog.NameComparator`. Instead, we'll do the following:

* Make the `NameComparator` class in `Dog.java` private

In [ ]:
private static class NameComparator implements Comparator<Dog>{
    ...
}

* Create a public static method `getNameComparator` that returns a Comparator object in `Dog.java`

In [ ]:
private static class NameComparator implements Comparator<Dog>{...}

public static Comparator<Dog> getNameComparator() {
    return new NameComparator();
}

Now back to the `main` method in `DogLauncher.java`,
* Change `new.DogNameComparator` to `Dog.getNameComparator()`.

In [ ]:
Dog.NameComparator nc = Dog.getNameComparator();

Now `Dog.NameComparator` is a private class, so we need a container that can hold a dog comparator. We can do the following,

In [ ]:
java.util.Comparator<Dog> nc = Dog.getNameComparator();

Since `java.util.Comparator` look verbose, we can import the `java.util.Comparator` in the beginning of the file so that we can jsut use `Comparator`,

In [ ]:
import java.util.Comparator;

public class DogLauncher {
    public static void main(String[] args){
        ...
        Comparator<Dog> nc = Dog.getNameComparator();
        ...
    }
}

## Example: `NameComparator`

Thus finally, we ended up with the following code,

<img src = 'namecomparator.png' width = 600/>

## Dogs and Comparator

The concluding schematic is as the following,

<img src = 'schematic.png' width = 600/>

We have a `Comparator<T>` interface that's built-in to Java
* We have `NameComparator` that we just built as a private class that implements `Comparator<T>` interface
* We could have created `SizeComparator` too, but we don't do it for the sake of lecture time

Note that when it comes to inheritance, the `Dog` class is not part of the tree. 

## Comparable and Comparator Summary

Interfaces provide us with the ability to make **callbacks**
* Sometimes a function needs the help of another function that might not have been written yet
    * Example: `max` needs `compareTo`
    * The helping function is sometimes called a `callback`
* Some languages handle this using explicit function passing
* In Java, we do this by wrapping up the needed function in an interface
    * e.g. `Arrays.sort` needs `compare` which lives inside the `comparator` interface
* `Arrays.sort` calls back whenever it needs a comparison
    * Similar to giving your numver to someone if they need information
    * See project 1B to explore how to write code that uses comparators
    
    
Extra facts: difference between `Comparable` and `Comparator`
* `Comparable`: making an object comparable with another object
* `Comparator`: compares 2 objects